In [58]:

import os
from dotenv import load_dotenv

In [59]:
load_dotenv()

True

In [60]:
yt_api_key = os.environ.get("YT_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [61]:
import os
from googleapiclient.discovery import build

# Create a YouTube Data API service
youtube = build("youtube", "v3", developerKey=yt_api_key)

# Search for videos
search_response = youtube.search().list(
    q="tomato soup recipe",
    type="video",
    part="id,snippet",
    maxResults=20
).execute()

# Print video titles and IDs
for item in search_response["items"]:
    print(f"Title: {item['snippet']['title']}, Video ID: {item['id']['videoId']}")


Title: Grilled Cheese and Tomato Soup, Video ID: PQ4yN68IbHc
Title: Tomato Garlic Soup, Video ID: 4vUlSHs33Ts
Title: Tomato Soup Recipe in 51 Seconds Restaurant Style, Video ID: NKO09-Rvkk4
Title: Grilled Cheese &amp; Roasted Tomato Soup, Video ID: NAalzSoefU8
Title: Creamy Tomato Soup | The Golden Balance, Video ID: s1YMjkPd2eo
Title: Tonight I’m craving soup and grilled cheese... and yes it&#39;s in my cookbook. #shorts, Video ID: wDVKIfQauWQ
Title: ROASTED TOMATO SOUP 🍅, Video ID: EAo3XMyxbg8
Title: tomato soup recipe | cream of tomato soup | टमाटर सूप रेसिपी | tomatoe soup recipe, Video ID: JjoaBoTWCUM
Title: The Best TOMATO SOUP RECIPE I Ever Made, Video ID: MkULY-TuRz0
Title: Tomato soup, Video ID: c2zyaaatkK4
Title: 😳 NO CREAM NO CORNFLOUR TOMATO 🍅 SOUP IN 10 mins | Easy Railway Style Tomato Soup | Healthy Soup, Video ID: gP4Ulh39Dpc
Title: Tomato soup recipe | #shorts | how to make tomato soup | kabitaskitchen, Video ID: BAhiVcS_f44
Title: होटल स्टाइल टमाटर का सूप बनाने की विधि

In [62]:
# Response Dict contains the following
# videoId, title, description

response_list = []
for item in search_response["items"]:
    response_dict = {
        "videoId": item["id"]["videoId"],
        "title": item["snippet"]["title"],
        "description": item["snippet"]["description"]
    }
    response_list.append(response_dict)

In [63]:
response_list

[{'videoId': 'PQ4yN68IbHc',
  'title': 'Grilled Cheese and Tomato Soup',
  'description': 'shorts #grilledcheese #tomatosoup.'},
 {'videoId': '4vUlSHs33Ts',
  'title': 'Tomato Garlic Soup',
  'description': 'Follow Toni: https://www.instagram.com/themoodyfoody.'},
 {'videoId': 'NKO09-Rvkk4',
  'title': 'Tomato Soup Recipe in 51 Seconds Restaurant Style',
  'description': 'Tomato Soup, must try recipe!'},
 {'videoId': 'NAalzSoefU8',
  'title': 'Grilled Cheese &amp; Roasted Tomato Soup',
  'description': "Good soup Fix those late night cravings with some bomb as grilled cheese and spicy tomato soup here's my recipe: 6 ..."},
 {'videoId': 's1YMjkPd2eo',
  'title': 'Creamy Tomato Soup | The Golden Balance',
  'description': 'shorts Creamy Tomato Soup 8 Roma tomatoes 1 onion 6-8 cloves of garlic 1 shallot 1 sprig of Rosemary and thyme Cover in ...'},
 {'videoId': 'wDVKIfQauWQ',
  'title': 'Tonight I’m craving soup and grilled cheese... and yes it&#39;s in my cookbook. #shorts',
  'descripti

In [64]:
from openai import OpenAI
import json
client = OpenAI(api_key=openai_api_key)

def get_yt_search(query):
    youtube = build("youtube", "v3", developerKey=yt_api_key)
    search_response = youtube.search().list(
        q=query,
        type="video",
        part="id,snippet",
        maxResults=10
    ).execute()

    response_list = []
    for item in search_response["items"]:
        response_dict = {
            "videoId": item["id"]["videoId"],
            "title": item["snippet"]["title"],
            "description": item["snippet"]["description"]
        }
        response_list.append(response_dict)

    return response_list

def get_relevant_yt_ids(query):

    response_list = get_yt_search(query)
    response_list_str = json.dumps(response_list)
    messages=[
        {"role": "system", "content": f"""You are helpful educational Assistant. 
         Given videoId, title and description of a youtube video, you need to find the relevant videoIds, for user query.
         Answer only top 5 videoIds from the given json data don't give videoId not in give data, in comma separated format, don't include any extra information.
         Below is the json data you have :
         {response_list_str}"""},
        {"role": "user", "content": query},
    ]
    print("messages", messages)
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
    )

    relevant_ids = response.choices[0].message.content

    return response_list, relevant_ids

In [65]:
response_list, video_ids = get_relevant_yt_ids("python lambda")

messages [{'role': 'system', 'content': 'You are helpful educational Assistant. \n         Given videoId, title and description of a youtube video, you need to find the relevant videoIds, for user query.\n         Answer only top 5 videoIds from the given json data don\'t give videoId not in give data, in comma separated format, don\'t include any extra information.\n         Below is the json data you have :\n         [{"videoId": "hYzwCsKGRrg", "title": "#42 Python Tutorial for Beginners | Anonymous Functions | Lambda", "description": "Spring and Microservices Live Course : https://bit.ly/springmslive Coupon: TELUSKO25 (25% Discount) Industry-Ready Java ..."}, {"videoId": "KR22jigJLok", "title": "Python Lambda Functions??", "description": "Join my Patreon: https://www.patreon.com/b001io Discord: https://discord.gg/jA8SShU8zJ Follow me on Twitter: ..."}, {"videoId": "25ovCm9jKfA", "title": "Lambda Expressions &amp; Anonymous Functions ||  Python Tutorial  ||  Learn Python Programming"

In [66]:
video_ids

'hYzwCsKGRrg, KR22jigJLok, 25ovCm9jKfA, P8MdDCTbMOI, UfFWf-PXUqE'

In [71]:
#youtube url 
video_ids_list = video_ids.split(",")
video_ids_list = [x.strip() for x in video_ids_list]
for videoId in video_ids_list:
    print("https://www.youtube.com/watch?v=" + videoId)

https://www.youtube.com/watch?v=hYzwCsKGRrg
https://www.youtube.com/watch?v=KR22jigJLok
https://www.youtube.com/watch?v=25ovCm9jKfA
https://www.youtube.com/watch?v=P8MdDCTbMOI
https://www.youtube.com/watch?v=UfFWf-PXUqE
